# DEMO 2

Regression applied to Stock Market Data


In [1]:
import sys
import os

# Manually set the path relative to the py file's location that you want to import
func_lib_path = os.path.abspath(os.path.join(os.getcwd(), '../../'))# Add the path to sys.path
sys.path.append(func_lib_path)

# Now you can import func_lib
import func_lib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
historical_prices = func_lib.createHistPrices()
list_of_momentums = [1, 5, 15, 20]
total_returns     = func_lib.computingReturns(historical_prices, list_of_momentums)
total_returns.dropna(inplace=True)

[*********************100%%**********************]  501 of 501 completed

1 Failed download:
['SW']: YFChartError("%ticker%: Data doesn't exist for startDate = 946702800, endDate = 1714536000")


In [3]:
total_returns.head()

F_1_d_returns  1_d_returns  5_d_returns  15_d_returns  \
Ticker Date                                                                
A      2000-02-01       0.077465     0.072710     0.048938      0.029919   
       2000-02-02       0.016339     0.077465     0.119854      0.125000   
       2000-02-03      -0.019293     0.016339     0.136073      0.166979   
       2000-02-04       0.042623    -0.019293     0.120294      0.127542   
       2000-02-07      -0.006290     0.042623     0.201133      0.162706   

                   20_d_returns  
Ticker Date                      
A      2000-02-01     -0.013889  
       2000-02-02      0.150376  
       2000-02-03      0.246493  
       2000-02-04      0.270834  
       2000-02-07      0.223077

In [4]:
# Determine the split index for 70% of the dates
unique_dates = total_returns.index.get_level_values('Date').unique()
split_date = unique_dates[int(0.7 * len(unique_dates))]
split_date

Timestamp('2017-01-20 00:00:00')

In [5]:
# Create the training set: all data before the split date
train_data = total_returns.loc[total_returns.index.get_level_values('Date') < split_date]

# Create the testing set: all data from the split date onwards
test_data = total_returns.loc[total_returns.index.get_level_values('Date') >= split_date]

In [6]:
total_returns  = test_data['F_1_d_returns']

In [7]:
features = ['1_d_returns', '5_d_returns', '15_d_returns', '20_d_returns']
target   = ['F_1_d_returns_Ind']

# Split the data into training and testing sets
X_train = train_data[features]
X_test  = test_data[features]
y_train = train_data[target]
y_test  = test_data[target]

KeyError: "None of [Index(['F_1_d_returns_Ind'], dtype='object')] are in the [columns]"

In [ ]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled  = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

In [ ]:
# Add a constant to the model (intercept)
X_train_scaled_const_add = sm.add_constant(X_train_scaled)
X_test_scaled_const_add  = sm.add_constant(X_test_scaled)

# Fit the model
# Creating and fitting the logistic regression model
model = sm.OLS(y_train, X_train_scaled_const_add).fit()

# Print the model summary
print(model.summary())

In [ ]:
model_name = 'Reg'
# Make predictions
y_pred = model.predict(X_test_scaled_const_add)

y_test_df = pd.DataFrame(y_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.rename(columns={0:model_name}, inplace=True)

y_test_and_pred = pd.merge(y_pred_df, total_returns, left_index=True, right_index=True)

In [ ]:
# Define trading strategy based on Logit
def trading_strategy(y_pred):
    if y_pred >0.5:
        return  1 # Go long
    else:
        return  0  


In [ ]:
cum_returns, calendar_returns = func_lib.compute_BM_Perf(y_test_and_pred)

In [ ]:
cum_returns, calendar_returns = func_lib.compute_strat_perf(y_test_and_pred, cum_returns, calendar_returns, trading_strategy, model_name)